Our endgoal is to both merge the timebudget exports and the aggregated events with both the playback plan and the manually collected metadata.

For that we need the path to these files. You can set them to paths as shown at the top of Windows explorer.

In [ ]:
import pandas as pd
import os

TIME_BUDGET_FOLDER = "data export/data_export_30_sec_cut_off/time budget/single xlsx"
AGGREGATED_EVENTS_FILE = "data export/data_export_30_sec_cut_off/aggregated events/aggregated_events_01.09.25.xlsx"
PLAYBACK_PLAN_FILE = "data export/playback_plan_filled_out.xlsx"
MANUAL_METADATA_FILE = "data export/playback_plan_BORIS_blind_coding_for analysis.xlsx"

OUTPUT_TIMEBUDGET = "output_timebudget.xlsx"
OUTPUT_AGGREGATED_EVENTS = "output_aggregated_events.xlsx"

Timebudgets are exported as indidivual files per observation id. Merge them all into one and add an ID keeping the origin.

These contain all behaviors for all subjects, even if the subject didn't exist in the specific observation or it didn't show the behavior. So we remove all entries where the "Total number of occurences" is 0.

There are some observations where two cameras where used, they have _a and _b suffixes that should be removed from the ID.

In [ ]:
# This is a function we can reuse later to do the same thing for the events
def strip_ab_and_create_unique_id(df: pd.DataFrame):
    df["Observation id"] = df["Observation id"].str.rstrip("_ab")
    df["uid"] = df["Observation id"].str.cat(df["Subject"], sep="_")
    return df

def remove_nonexistant(df: pd.DataFrame):
    return df[df["Total number of occurences"] > 0]

# Create an iterator over all files in the timebudget folder
files = map(lambda x: os.path.join(TIME_BUDGET_FOLDER, x), os.listdir(TIME_BUDGET_FOLDER))
# Open them as excel files
excels = map(pd.read_excel, files)
# Apply the function to all
without_nonexistant = map(remove_nonexistant, excels)
# Again apply the function to all
with_ids = map(strip_ab_and_create_unique_id, without_nonexistant)

# Concatenate all individual files
merged_timebudget = pd.concat(with_ids)
merged_timebudget.set_index("Observation id")
merged_timebudget

From the aggregated events export we need to remove any rows where there is no subject, then pivot the table on the Behavior columns, counting the occurences.

In [ ]:
aggregated_events = pd.read_excel(AGGREGATED_EVENTS_FILE)
agg = strip_ab_and_create_unique_id(aggregated_events)
agg = agg[agg["Subject"] != "No focal subject"]

# This shows the table before pivoting, if you remove the #
# display(agg)

counted_events = agg.pivot_table(index=["uid", "Observation id", "Subject"], columns="Behavior", aggfunc="size", fill_value=0)
counted_events

The playback plan contains lots of data, but we are currently only interested in the stimulus category and stimulus. These we need to match on the Observation id, so we also create one here.

In [ ]:
playback_plan = pd.read_excel(PLAYBACK_PLAN_FILE)

padded_experiment_number = playback_plan["experiment number"].astype(str).str.zfill(2)

playback_plan["Observation id"] = padded_experiment_number.str.cat(playback_plan["date"].dt.strftime("%d.%m.%Y"), sep="_")
playback_plan = playback_plan.set_index("Observation id")
playback_plan = playback_plan[["stim. cat.", "stimulus"]]
playback_plan


Handwritten metadata has one row per observation. So it contains data belonging to the observation itself and the individual focals. The tricky part is the way that this is organized, as we have columns name "focal 1" through "focal 5", which contain the values for said subjects.

In [ ]:
# These need to be merged onto all obervations
unspecific_columns = [
    "experiment number",
    "date",
    "time",
    "location (latitude)",
    "location (longitude)",
    "species",
    "groupsize category",
    "other species present",
    "group comp.",
    "habitat",
    "group 30sec-looking",
    "group 30sec-moving",
    "temp",
    "wind",
    "dist. to speaker",
    "car side"
]

manual_metadata = pd.read_excel(MANUAL_METADATA_FILE)

# Also strip _ab here
manual_metadata["Observation id"] = manual_metadata["Observation id"].str.rstrip("_ab")
manual_metadata = manual_metadata.set_index("Observation id")

# This function will extract the relevant columns for subject n
def focal_specific(n):
    df = manual_metadata[[f"focal {n}", f"f{n} move of"]]
    df = df.rename(columns={f"focal {n}": "sex", f"f{n} move of": "move of distance"})
    df = df[df["sex"] != "/"]
    df = df.dropna()
    df["Subject"] = f"Focal {n}"
    return df

# Create a single table with the extracted specifics for focals 1 to 5 (the end of range is exclusive)
focal_metadata = pd.concat([focal_specific(n) for n in range(1, 6)]).reset_index()
display(focal_metadata)

unspecific_metadata = manual_metadata[unspecific_columns]
unspecific_metadata

Now we put everything together

In [ ]:
def merge_with_data(df: pd.DataFrame):
    with_plan = df.join(other=playback_plan)
    with_meta = with_plan.join(other=unspecific_metadata)
    return pd.merge(left=with_meta, right=focal_metadata, on=["Observation id", "Subject"], how="left")

full_timebudget = merge_with_data(merged_timebudget)
display(full_timebudget)

full_counted_events = merge_with_data(counted_events)
full_counted_events


In [ ]:
full_timebudget.to_excel(OUTPUT_TIMEBUDGET)
full_counted_events.to_excel(OUTPUT_AGGREGATED_EVENTS)